![Kayak](https://seekvectorlogo.com/wp-content/uploads/2018/01/kayak-vector-logo.png)

# Plan your trip with Kayak 

# 00/ import lib

In [4]:
import pandas as pd 
import numpy as np 
import plotly.express as px

from geopy.geocoders import Nominatim
from meteofrance_api import MeteoFranceClient
from datetime import datetime

import scrapy
import json

import boto3
import dotenv

import requests
import sqlalchemy

## 1.2/ goals of project

## Goals 🎯

As the project has just started, your team doesn't have any data that can be used to create this application. Therefore, your job will be to: 

* Scrape data from destinations 
* Get weather data from each destination 
* Get hotels' info about each destination
* Store all the information above in a data lake
* Extract, transform and load cleaned data from your datalake to a data warehouse

## Scope of this project 🖼️

Marketing team wants to focus first on the best cities to travel to in France. According <a href="https://one-week-in.com/35-cities-to-visit-in-france/" target="_blank">One Week In.com</a> here are the top-35 cities to visit in France: 

```python 
["Mont Saint Michel",
"St Malo",
"Bayeux",
"Le Havre",
"Rouen",
"Paris",
"Amiens",
"Lille",
"Strasbourg",
"Chateau du Haut Koenigsbourg",
"Colmar",
"Eguisheim",
"Besancon",
"Dijon",
"Annecy",
"Grenoble",
"Lyon",
"Gorges du Verdon",
"Bormes les Mimosas",
"Cassis",
"Marseille",
"Aix en Provence",
"Avignon",
"Uzes",
"Nimes",
"Aigues Mortes",
"Saintes Maries de la mer",
"Collioure",
"Carcassonne",
"Ariege",
"Toulouse",
"Montauban",
"Biarritz",
"Bayonne",
"La Rochelle"]
```

Your team should focus **only on the above cities for your project**. 


## Helpers 🦮

To help you achieve this project, here are a few tips that should help you

### Get weather data with an API 

*   Use https://nominatim.org/ to get the gps coordinates of all the cities (no subscription required) Documentation : https://nominatim.org/release-docs/develop/api/Search/

*   Use https://openweathermap.org/appid (you have to subscribe to get a free apikey) and https://openweathermap.org/api/one-call-api to get some information about the weather for the 35 cities and put it in a DataFrame

*   Determine the list of cities where the weather will be the nicest within the next 7 days For example, you can use the values of daily.pop and daily.rain to compute the expected volume of rain within the next 7 days... But it's only an example, actually you can have different opinions on a what a nice weather would be like 😎 Maybe the most important criterion for you is the temperature or humidity, so feel free to change the rules !

*   Save all the results in a `.csv` file, you will use it later 😉 You can save all the informations that seem important to you ! Don't forget to save the name of the cities, and also to create a column containing a unique identifier (id) of each city (this is important for what's next in the project)

*   Use plotly to display the best destinations on a map

### Scrape Booking.com 

Since BookingHoldings doesn't have aggregated databases, it will be much faster to scrape data directly from booking.com 

You can scrap as many information asyou want, but we suggest that you get at least:

*   hotel name,
*   Url to its booking.com page,
*   Its coordinates: latitude and longitude
*   Score given by the website users
*   Text description of the hotel


### Create your data lake using S3 

Once you managed to build your dataset, you should store into S3 as a csv file. 

### ETL 

Once you uploaded your data onto S3, it will be better for the next data analysis team to extract clean data directly from a Data Warehouse. Therefore, create a SQL Database using AWS RDS, extract your data from S3 and store it in your newly created DB. 

## résumé

scrap booking + call API meteo 

un df   :
*        => nom de destination [=> (sort by meteo and notation) ]
*        => url booking 
*        => coordonnées (long lat) scrap booking
*        => coordonées (long lat) meteo 
*        => user score
*        => description 
*        => meteo (temps) (a j et j+7) (donc variation) 
*        => meteo (temperature) (a j et j+7) (donc variation)

df => to csv => to Bucket S3    
S3 => SQL => notebook => plotly top 5 top 20

# 01/ list of place 

In [37]:
holy_place = [
"Mont Saint Michel",
"St Malo",
"Bayeux",
"Le Havre",
"Rouen",
"Paris",
"Amiens",
"Lille",
"Strasbourg",
"Chateau du Haut Koenigsbourg",
"Colmar",
"Eguisheim",
"Besancon",
"Dijon",
"Annecy",
"Grenoble",
"Lyon",
"Gorges du Verdon",
"Bormes les Mimosas",
"Cassis",
"Marseille",
"Aix en Provence",
"Avignon",
"Uzes",
"Nimes",
"Aigues Mortes",
"Saintes Maries de la mer",
"Collioure",
"Carcassonne",
"Ariege",
"Toulouse",
"Montauban",
"Biarritz",
"Bayonne",
"La Rochelle"
]

In [39]:
len(holy_place)

35

# 02/ API get informations for the place 

### 2.1/ lat long & ID

In [4]:
from geopy.geocoders import Nominatim

# Initialiser un géocodeur basé sur OpenStreetMap
geolocator = Nominatim(timeout=10, user_agent="meteo_app")

In [5]:
cities = ["Paris, France", "Lyon, France", "Marseille, France"]

In [6]:
for city in cities:
    location = geolocator.geocode(f'{city}, France')
    if location:
        print(f"{city} → lat: {location.latitude}, lon: {location.longitude}")
    else:
        print(f"❌ Impossible de trouver {city}")

Paris, France → lat: 48.8588897, lon: 2.320041
Lyon, France → lat: 45.7578137, lon: 4.8320114
Marseille, France → lat: 43.2961743, lon: 5.3699525


In [7]:
for city in holy_place:
    location = geolocator.geocode(f'{city}, France')
    if location:
        print(f"{city} → lat: {location.latitude}, lon: {location.longitude}")
    else:
        print(f"❌ Impossible de trouver {city}")

Mont Saint Michel → lat: 48.6359541, lon: -1.51146
St Malo → lat: 48.649518, lon: -2.0260409
Bayeux → lat: 49.2764624, lon: -0.7024738
Le Havre → lat: 49.4938975, lon: 0.1079732
Rouen → lat: 49.4404591, lon: 1.0939658
Paris → lat: 48.8534951, lon: 2.3483915
Amiens → lat: 49.8941708, lon: 2.2956951
Lille → lat: 50.6365654, lon: 3.0635282
Strasbourg → lat: 48.584614, lon: 7.7507127
Chateau du Haut Koenigsbourg → lat: 48.2494107, lon: 7.3443202
Colmar → lat: 48.0777517, lon: 7.3579641
Eguisheim → lat: 48.0447968, lon: 7.3079618
Besancon → lat: 47.2380222, lon: 6.0243622
Dijon → lat: 47.3215806, lon: 5.0414701
Annecy → lat: 45.8992348, lon: 6.1288847
Grenoble → lat: 45.1875602, lon: 5.7357819
Lyon → lat: 45.7578137, lon: 4.8320114
Gorges du Verdon → lat: 43.7496562, lon: 6.3285616
Bormes les Mimosas → lat: 43.1506968, lon: 6.3419285
Cassis → lat: 43.2140359, lon: 5.5396318
Marseille → lat: 43.2961743, lon: 5.3699525
Aix en Provence → lat: 43.5298424, lon: 5.4474738
Avignon → lat: 43.949249

In [8]:
from meteofrance_api import MeteoFranceClient

client = MeteoFranceClient()

In [9]:
for city in cities:
    location = geolocator.geocode(f'{city}, France')
    if location:
        print(f"{city} → lat: {location.latitude}, lon: {location.longitude}")
        forcast_test = client.get_forecast(location.latitude, location.longitude)
        id = forcast_test.position
        print(f'code insee => {id}')
    else:
        print(f"❌ Impossible de trouver {city}")

Paris, France → lat: 48.8588897, lon: 2.320041
code insee => {'lat': 48.856636, 'lon': 2.320161, 'alti': 35, 'name': 'Paris—7ème Arrondissement', 'country': 'FR - France', 'dept': '75', 'rain_product_available': 1, 'timezone': 'Europe/Paris', 'insee': '751070', 'bulletin_cote': 0}
Lyon, France → lat: 45.7578137, lon: 4.8320114
code insee => {'lat': 45.758097, 'lon': 4.8407, 'alti': 169, 'name': 'Lyon', 'country': 'FR - France', 'dept': '69', 'rain_product_available': 1, 'timezone': 'Europe/Paris', 'insee': '691230', 'bulletin_cote': 0}
Marseille, France → lat: 43.2961743, lon: 5.3699525
code insee => {'lat': 43.296199, 'lon': 5.375945, 'alti': 5, 'name': 'Marseille', 'country': 'FR - France', 'dept': '13', 'rain_product_available': 1, 'timezone': 'Europe/Paris', 'insee': '130550', 'bulletin_cote': 1}


In [10]:
for city in cities:
    location = geolocator.geocode(f'{city}, France')
    if location:
        print(f"{city} → lat: {location.latitude}, lon: {location.longitude}")
        forcast_test = client.get_forecast(location.latitude, location.longitude)
        id = forcast_test.position['insee']
        print(f'code insee => {id}')
    else:
        print(f"❌ Impossible de trouver {city}")

Paris, France → lat: 48.8588897, lon: 2.320041
code insee => 751070
Lyon, France → lat: 45.7578137, lon: 4.8320114
code insee => 691230
Marseille, France → lat: 43.2961743, lon: 5.3699525
code insee => 130550


### 2.2/ Weather

In [11]:
from geopy.geocoders import Nominatim
from meteofrance_api import MeteoFranceClient
from datetime import datetime

In [12]:
# Initialiser geocoder + client météo
geolocator = Nominatim(timeout=10, user_agent="meteo_app")
client = MeteoFranceClient()

In [13]:
def get_forecast_by_city(city_name):
    """Retourne les prévisions J→J+2 pour une ville donnée"""
    # 1. Géocodage
    location = geolocator.geocode(f'{city_name}, France')
    if not location:
        return city_name, None

    # 2. Récupérer les prévisions météo + id
    forecast = client.get_forecast(location.latitude, location.longitude)
    
    # 3. Extraire les 2 prochains jours
    daily = forecast.daily_forecast[:2]
    id_place = forecast.position['insee']
    return city_name, daily, id_place

In [14]:
# Liste de villes/lieux test
places = ["Paris, France", "Lyon, France", "Marseille, France"]

In [15]:
# Boucler sur la liste
for place in places:
    city, daily_forecast, id_place = get_forecast_by_city(f'{place}, France')
    if daily_forecast:
        print(f"🌍 Prévisions pour {city} :")
        print(f'unique insee id {id_place} \n')
        location = geolocator.geocode(f'{place}, France')
        print(f"→ lat: {location.latitude}, lon: {location.longitude} \n")
        for day in daily_forecast:
            date = datetime.fromtimestamp(day["dt"]).strftime("%Y-%m-%d")
            tmin = day["T"]["min"]
            tmax = day["T"]["max"]
            print(f" - {date}: {tmin}°C → {tmax}°C")
        print()
    else:
        print(f"❌ Impossible de trouver {city}")


🌍 Prévisions pour Paris, France, France :
unique insee id 751070 

→ lat: 48.8588897, lon: 2.320041 

 - 2025-09-30: 10.1°C → 21°C
 - 2025-10-01: 10.4°C → 20.1°C

🌍 Prévisions pour Lyon, France, France :
unique insee id 691230 

→ lat: 45.7578137, lon: 4.8320114 

 - 2025-09-30: 11°C → 19.6°C
 - 2025-10-01: 12°C → 20.2°C

🌍 Prévisions pour Marseille, France, France :
unique insee id 130550 

→ lat: 43.2961743, lon: 5.3699525 

 - 2025-09-30: 12.7°C → 23.6°C
 - 2025-10-01: 13.9°C → 23°C



In [16]:
# Liste de villes/lieux
# places = ["Paris, France", "Lyon, France", "Marseille, France"]
holy_place

['Mont Saint Michel',
 'St Malo',
 'Bayeux',
 'Le Havre',
 'Rouen',
 'Paris',
 'Amiens',
 'Lille',
 'Strasbourg',
 'Chateau du Haut Koenigsbourg',
 'Colmar',
 'Eguisheim',
 'Besancon',
 'Dijon',
 'Annecy',
 'Grenoble',
 'Lyon',
 'Gorges du Verdon',
 'Bormes les Mimosas',
 'Cassis',
 'Marseille',
 'Aix en Provence',
 'Avignon',
 'Uzes',
 'Nimes',
 'Aigues Mortes',
 'Saintes Maries de la mer',
 'Collioure',
 'Carcassonne',
 'Ariege',
 'Toulouse',
 'Montauban',
 'Biarritz',
 'Bayonne',
 'La Rochelle']

In [17]:
# Boucler sur la liste
for place in holy_place:
    city, daily_forecast, id_place = get_forecast_by_city(f'{place}, France')
    if daily_forecast:
        print(f"🌍 Prévisions pour {city} :")
        print(f'unique insee id {id_place} \n')
        location = geolocator.geocode(f'{place}, France')
        print(f"→ lat: {location.latitude}, lon: {location.longitude} \n")
        for day in daily_forecast:
            date = datetime.fromtimestamp(day["dt"]).strftime("%Y-%m-%d")
            tmin = day["T"]["min"]
            tmax = day["T"]["max"]
            print(f" - {date}: {tmin}°C → {tmax}°C")
        print()
    else:
        print(f"❌ Impossible de trouver {city}")

🌍 Prévisions pour Mont Saint Michel, France :
unique insee id 5035351 

→ lat: 48.6359541, lon: -1.51146 

 - 2025-09-30: 8.9°C → 19.6°C
 - 2025-10-01: 7.2°C → 20.9°C

🌍 Prévisions pour St Malo, France :
unique insee id 3528885 

→ lat: 48.649518, lon: -2.0260409 

 - 2025-09-30: 12°C → 19.5°C
 - 2025-10-01: 8.2°C → 22.1°C

🌍 Prévisions pour Bayeux, France :
unique insee id 140470 

→ lat: 49.2764624, lon: -0.7024738 

 - 2025-09-30: 7°C → 19.1°C
 - 2025-10-01: 8.2°C → 20.6°C

🌍 Prévisions pour Le Havre, France :
unique insee id 763510 

→ lat: 49.4938975, lon: 0.1079732 

 - 2025-09-30: 9.9°C → 19.1°C
 - 2025-10-01: 8.5°C → 20.6°C

🌍 Prévisions pour Rouen, France :
unique insee id 765400 

→ lat: 49.4404591, lon: 1.0939658 

 - 2025-09-30: 8.4°C → 20.4°C
 - 2025-10-01: 6.9°C → 21.9°C

🌍 Prévisions pour Paris, France :
unique insee id 751070 

→ lat: 48.8534951, lon: 2.3483915 

 - 2025-09-30: 10.1°C → 21°C
 - 2025-10-01: 10.4°C → 20.1°C

🌍 Prévisions pour Amiens, France :
unique insee

In [18]:
daily_forecast

[{'dt': 1759190400,
  'T': {'min': 10.2, 'max': 22, 'sea': None},
  'humidity': {'min': 45, 'max': 90},
  'precipitation': {'24h': 0},
  'uv': 4,
  'weather12H': {'icon': 'p1j', 'desc': 'Ciel clair'},
  'sun': {'rise': 1759212067, 'set': 1759254445}},
 {'dt': 1759276800,
  'T': {'min': 10.5, 'max': 22, 'sea': None},
  'humidity': {'min': 40, 'max': 85},
  'precipitation': {'24h': 0},
  'uv': 3,
  'weather12H': {'icon': 'p4j', 'desc': 'Ciel voilé'},
  'sun': {'rise': 1759298545, 'set': 1759340729}}]

In [19]:
for place in places:
    city, daily_forecast, id_place = get_forecast_by_city(f'{place}, France')
    if daily_forecast:
        print(f"🌍 Prévisions pour {city} :")
        print(f'unique insee id {id_place} \n')
        location = geolocator.geocode(f'{place}, France')
        print(f"→ lat: {location.latitude}, lon: {location.longitude} \n")
        for day in daily_forecast:
            date = datetime.fromtimestamp(day["dt"]).strftime("%Y-%m-%d, %H:%M")
            tmin = day["T"]["min"]
            tmax = day["T"]["max"]
            pluie = day['precipitation']['24h']
            uv = day['uv']
            meteo = day['weather12H']['desc']
            print(f" - {date}: {tmin}°C → {tmax}°C \n pluie => {pluie} uv => {uv} meteo => {meteo}")
        print()
    else:
        print(f"❌ Impossible de trouver {city}")


🌍 Prévisions pour Paris, France, France :
unique insee id 751070 

→ lat: 48.8588897, lon: 2.320041 

 - 2025-09-30, 02:00: 10.1°C → 21°C 
 pluie => 0 uv => 3 meteo => Ciel clair
 - 2025-10-01, 02:00: 10.4°C → 20.1°C 
 pluie => 0 uv => 3 meteo => Ciel voilé

🌍 Prévisions pour Lyon, France, France :
unique insee id 691230 

→ lat: 45.7578137, lon: 4.8320114 

 - 2025-09-30, 02:00: 11°C → 19.6°C 
 pluie => 0 uv => 4 meteo => Eclaircies
 - 2025-10-01, 02:00: 12°C → 20.2°C 
 pluie => 0 uv => 4 meteo => Ciel clair

🌍 Prévisions pour Marseille, France, France :
unique insee id 130550 

→ lat: 43.2961743, lon: 5.3699525 

 - 2025-09-30, 02:00: 12.7°C → 23.6°C 
 pluie => 0.1 uv => 4 meteo => Ciel clair
 - 2025-10-01, 02:00: 13.9°C → 23°C 
 pluie => 0 uv => 4 meteo => Ciel clair



In [20]:
for place in holy_place:
    city, daily_forecast, id_place = get_forecast_by_city(f'{place}, France')
    if daily_forecast:
        print(f"🌍 Prévisions pour {city} :")
        location = geolocator.geocode(f'{place}, France')
        print(f"→ lat: {location.latitude}, lon: {location.longitude} \n")
        for day in daily_forecast:
            date = datetime.fromtimestamp(day["dt"]).strftime("%Y-%m-%d")
            tmin = day["T"]["min"]
            tmax = day["T"]["max"]
            pluie = day['precipitation']['24h']
            uv = day['uv']
            meteo = day['weather12H']['desc']
            id = day['weather12H']['desc']
            print(f" - {date}: {tmin}°C → {tmax}°C \n pluie => {pluie} uv => {uv} meteo => {meteo}")
        print()
    else:
        print(f"❌ Impossible de trouver {city}")


🌍 Prévisions pour Mont Saint Michel, France :
→ lat: 48.6359541, lon: -1.51146 

 - 2025-09-30: 8.9°C → 19.6°C 
 pluie => 0.2 uv => 3 meteo => Ciel clair
 - 2025-10-01: 7.2°C → 20.9°C 
 pluie => 0 uv => 2 meteo => Ciel voilé

🌍 Prévisions pour St Malo, France :
→ lat: 48.649518, lon: -2.0260409 

 - 2025-09-30: 12°C → 19.5°C 
 pluie => 0 uv => 3 meteo => Bancs de Brouillard
 - 2025-10-01: 8.2°C → 22.1°C 
 pluie => 0 uv => 2 meteo => Eclaircies

🌍 Prévisions pour Bayeux, France :
→ lat: 49.2764624, lon: -0.7024738 

 - 2025-09-30: 7°C → 19.1°C 
 pluie => 0 uv => 3 meteo => Ciel clair
 - 2025-10-01: 8.2°C → 20.6°C 
 pluie => 0 uv => 2 meteo => Eclaircies

🌍 Prévisions pour Le Havre, France :
→ lat: 49.4938975, lon: 0.1079732 

 - 2025-09-30: 9.9°C → 19.1°C 
 pluie => 0 uv => 3 meteo => Ciel clair
 - 2025-10-01: 8.5°C → 20.6°C 
 pluie => 0 uv => 2 meteo => Eclaircies

🌍 Prévisions pour Rouen, France :
→ lat: 49.4404591, lon: 1.0939658 

 - 2025-09-30: 8.4°C → 20.4°C 
 pluie => 0 uv => 3 m

### 2.3/ df create

In [21]:
places

['Paris, France', 'Lyon, France', 'Marseille, France']

In [22]:
holy_place

['Mont Saint Michel',
 'St Malo',
 'Bayeux',
 'Le Havre',
 'Rouen',
 'Paris',
 'Amiens',
 'Lille',
 'Strasbourg',
 'Chateau du Haut Koenigsbourg',
 'Colmar',
 'Eguisheim',
 'Besancon',
 'Dijon',
 'Annecy',
 'Grenoble',
 'Lyon',
 'Gorges du Verdon',
 'Bormes les Mimosas',
 'Cassis',
 'Marseille',
 'Aix en Provence',
 'Avignon',
 'Uzes',
 'Nimes',
 'Aigues Mortes',
 'Saintes Maries de la mer',
 'Collioure',
 'Carcassonne',
 'Ariege',
 'Toulouse',
 'Montauban',
 'Biarritz',
 'Bayonne',
 'La Rochelle']

In [ ]:
holy_place.value

In [23]:
def get_forecast_by_city_fordf(city_name):
    """Retourne les prévisions J→J+7 pour une ville donnée"""
    # 1. Géocodage
    location = geolocator.geocode(f'{city_name}, France')
    if not location:
        return city_name, None

    # 2. Récupérer les prévisions météo + id
    forecast = client.get_forecast(location.latitude, location.longitude)
    
    # 3. Extraire les 7 prochains jours
    daily = forecast.daily_forecast[:7]
    id_place = forecast.position['insee']
    return city_name, daily, id_place, location.latitude, location.longitude

In [24]:
data_weather_test = []
for place in places:
    city, daily_forecast, id_place, lat, lon = get_forecast_by_city_fordf(place)
    if daily_forecast:
        for day in daily_forecast:
            row = {
                "insee": id_place,                
                "ville": city,
                "lat": lat,
                "lon": lon,
                "date": datetime.fromtimestamp(day["dt"]).strftime("%Y-%m-%d"),
                "tmin": day["T"]["min"],
                "tmax": day["T"]["max"],
                "pluie": day['precipitation']['24h'],
                "uv": day['uv'],
                "meteo": day['weather12H']['desc']
            }
            data_weather_test.append(row)

df_weather_test = pd.DataFrame(data_weather_test)
df_weather_test

,insee,ville,lat,lon,date,tmin,tmax,pluie,uv,meteo
0,751070,"Paris, France",48.858890,2.320041,2025-09-30,10.1,21.0,0.0,3.0,Ciel clair
1,751070,"Paris, France",48.858890,2.320041,2025-10-01,10.4,20.1,0.0,3.0,Ciel voilé
2,751070,"Paris, France",48.858890,2.320041,2025-10-02,10.7,19.6,0.0,2.0,Eclaircies
3,751070,"Paris, France",48.858890,2.320041,2025-10-03,11.6,19.6,6.8,2.0,Pluie faible
4,751070,"Paris, France",48.858890,2.320041,2025-10-04,11.2,15.6,15.8,NaN,Averses
5,751070,"Paris, France",48.858890,2.320041,2025-10-05,10.6,15.1,1.5,NaN,Averses
6,751070,"Paris, France",48.858890,2.320041,2025-10-06,10.5,17.9,0.4,NaN,Pluie
7,691230,"Lyon, France",45.757814,4.832011,2025-09-30,11.0,19.6,0.0,4.0,Eclaircies
8,691230,"Lyon, France",45.757814,4.832011,2025-10-01,12.0,20.2,0.0,4.0,Ciel clair
9,691230,"Lyon, France",45.757814,4.832011,2025-10-02,9.1,19.4,0.0,4.0,Ciel clair


In [25]:
data_weather = []
for place in holy_place:
    city, daily_forecast, id_place, lat, lon = get_forecast_by_city_fordf(place)
    if daily_forecast:
        for day in daily_forecast:
            row = {
                "insee": id_place,                
                "ville": city,
                "lat": lat,
                "lon": lon,
                "date": datetime.fromtimestamp(day["dt"]).strftime("%Y-%m-%d"),
                "tmin": day["T"]["min"],
                "tmax": day["T"]["max"],
                "pluie": day['precipitation']['24h']
            }
            data_weather.append(row)

df_weather = pd.DataFrame(data_weather)
df_weather

,insee,ville,lat,lon,date,tmin,tmax,pluie
0,5035351,Mont Saint Michel,48.635954,-1.511460,2025-09-30,8.9,19.6,0.2
1,5035351,Mont Saint Michel,48.635954,-1.511460,2025-10-01,7.2,20.9,0.0
2,5035351,Mont Saint Michel,48.635954,-1.511460,2025-10-02,8.4,19.7,0.0
3,5035351,Mont Saint Michel,48.635954,-1.511460,2025-10-03,12.2,20.1,10.7
4,5035351,Mont Saint Michel,48.635954,-1.511460,2025-10-04,11.4,16.5,19.3
...,...,...,...,...,...,...,...,...
240,173000,La Rochelle,46.159732,-1.151595,2025-10-02,10.5,22.0,0.0
241,173000,La Rochelle,46.159732,-1.151595,2025-10-03,12.6,20.9,0.0
242,173000,La Rochelle,46.159732,-1.151595,2025-10-04,14.7,18.5,9.8
243,173000,La Rochelle,46.159732,-1.151595,2025-10-05,13.6,17.2,1.4


In [ ]:
# pd.DataFrame.to_csv(df_weather, "data/weather.csv", index=False)

# 03/ first clean and visualisation

In [19]:
import pandas as pd 
import numpy as np 

In [20]:
try:
    # essayer d'utiliser un df déjà existant
    df_weather
    print("✅ DataFrame déjà disponible en mémoire")

except NameError:
    # si df n'existe pas → on le lit depuis le CSV
    print("⚠️ DataFrame introuvable → import depuis CSV")
    df_weather = pd.read_csv("data/weather.csv")
    print(df_weather.info())

⚠️ DataFrame introuvable → import depuis CSV
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 245 entries, 0 to 244
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   insee   245 non-null    int64  
 1   ville   245 non-null    object 
 2   lat     245 non-null    float64
 3   lon     245 non-null    float64
 4   date    245 non-null    object 
 5   tmin    245 non-null    float64
 6   tmax    245 non-null    float64
 7   pluie   245 non-null    float64
dtypes: float64(5), int64(1), object(2)
memory usage: 15.4+ KB
None


In [21]:
df_weather.head(7)

,insee,ville,lat,lon,date,tmin,tmax,pluie
0,5035351,Mont Saint Michel,48.635954,-1.51146,2025-09-30,8.9,19.6,0.2
1,5035351,Mont Saint Michel,48.635954,-1.51146,2025-10-01,7.2,20.9,0.0
2,5035351,Mont Saint Michel,48.635954,-1.51146,2025-10-02,8.4,19.7,0.0
3,5035351,Mont Saint Michel,48.635954,-1.51146,2025-10-03,12.2,20.1,10.7
4,5035351,Mont Saint Michel,48.635954,-1.51146,2025-10-04,11.4,16.5,19.3
5,5035351,Mont Saint Michel,48.635954,-1.51146,2025-10-05,10.6,16.4,3.9
6,5035351,Mont Saint Michel,48.635954,-1.51146,2025-10-06,10.2,17.6,3.0


In [22]:
df_weather['td_mean'] = np.mean(df_weather[['tmin', 'tmax']], axis=1)
df_weather

,insee,ville,lat,lon,date,tmin,tmax,pluie,td_mean
0,5035351,Mont Saint Michel,48.635954,-1.511460,2025-09-30,8.9,19.6,0.2,14.25
1,5035351,Mont Saint Michel,48.635954,-1.511460,2025-10-01,7.2,20.9,0.0,14.05
2,5035351,Mont Saint Michel,48.635954,-1.511460,2025-10-02,8.4,19.7,0.0,14.05
3,5035351,Mont Saint Michel,48.635954,-1.511460,2025-10-03,12.2,20.1,10.7,16.15
4,5035351,Mont Saint Michel,48.635954,-1.511460,2025-10-04,11.4,16.5,19.3,13.95
...,...,...,...,...,...,...,...,...,...
240,173000,La Rochelle,46.159732,-1.151595,2025-10-02,10.5,22.0,0.0,16.25
241,173000,La Rochelle,46.159732,-1.151595,2025-10-03,12.6,20.9,0.0,16.75
242,173000,La Rochelle,46.159732,-1.151595,2025-10-04,14.7,18.5,9.8,16.60
243,173000,La Rochelle,46.159732,-1.151595,2025-10-05,13.6,17.2,1.4,15.40


In [23]:
df_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 245 entries, 0 to 244
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   insee    245 non-null    int64  
 1   ville    245 non-null    object 
 2   lat      245 non-null    float64
 3   lon      245 non-null    float64
 4   date     245 non-null    object 
 5   tmin     245 non-null    float64
 6   tmax     245 non-null    float64
 7   pluie    245 non-null    float64
 8   td_mean  245 non-null    float64
dtypes: float64(6), int64(1), object(2)
memory usage: 17.4+ KB


In [42]:
len(df_weather)

245

In [44]:
len(df_weather['ville'].value_counts())

35

In [24]:
df_w = df_weather.copy(deep=True)

In [25]:
df_w['pluiew_mean'] = df_w.groupby('insee')['pluie'].transform("mean")

In [26]:
df_w['tw_mean'] = df_w.groupby('insee')['td_mean'].transform("mean")

In [27]:
df_w.head(10)

,insee,ville,lat,lon,date,tmin,tmax,pluie,td_mean,pluiew_mean,tw_mean
0,5035351,Mont Saint Michel,48.635954,-1.511460,2025-09-30,8.9,19.6,0.2,14.25,5.300000,14.264286
1,5035351,Mont Saint Michel,48.635954,-1.511460,2025-10-01,7.2,20.9,0.0,14.05,5.300000,14.264286
2,5035351,Mont Saint Michel,48.635954,-1.511460,2025-10-02,8.4,19.7,0.0,14.05,5.300000,14.264286
3,5035351,Mont Saint Michel,48.635954,-1.511460,2025-10-03,12.2,20.1,10.7,16.15,5.300000,14.264286
4,5035351,Mont Saint Michel,48.635954,-1.511460,2025-10-04,11.4,16.5,19.3,13.95,5.300000,14.264286
5,5035351,Mont Saint Michel,48.635954,-1.511460,2025-10-05,10.6,16.4,3.9,13.50,5.300000,14.264286
6,5035351,Mont Saint Michel,48.635954,-1.511460,2025-10-06,10.2,17.6,3.0,13.90,5.300000,14.264286
7,3528885,St Malo,48.649518,-2.026041,2025-09-30,12.0,19.5,0.0,15.75,3.771429,15.350000
8,3528885,St Malo,48.649518,-2.026041,2025-10-01,8.2,22.1,0.0,15.15,3.771429,15.350000
9,3528885,St Malo,48.649518,-2.026041,2025-10-02,10.4,20.5,0.0,15.45,3.771429,15.350000


In [45]:
len(df_w['ville'].value_counts())

35

In [28]:
df_w.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 245 entries, 0 to 244
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   insee        245 non-null    int64  
 1   ville        245 non-null    object 
 2   lat          245 non-null    float64
 3   lon          245 non-null    float64
 4   date         245 non-null    object 
 5   tmin         245 non-null    float64
 6   tmax         245 non-null    float64
 7   pluie        245 non-null    float64
 8   td_mean      245 non-null    float64
 9   pluiew_mean  245 non-null    float64
 10  tw_mean      245 non-null    float64
dtypes: float64(8), int64(1), object(2)
memory usage: 21.2+ KB


In [29]:
df_w1 = df_w.groupby(by='insee').last().reset_index()

In [46]:
len(df_w1['ville'].value_counts())

35

In [30]:
df_w1.head()

,insee,ville,lat,lon,date,tmin,tmax,pluie,td_mean,pluiew_mean,tw_mean
0,41440,Gorges du Verdon,43.749656,6.328562,2025-10-06,1.0,17.1,0.0,9.05,0.400000,10.871429
1,91960,Ariege,42.945537,1.406554,2025-10-06,8.9,14.0,0.0,11.45,1.485714,13.085714
2,110690,Carcassonne,43.213036,2.349107,2025-10-06,11.1,19.7,0.2,15.40,0.385714,17.085714
3,130010,Aix en Provence,43.529842,5.447474,2025-10-06,8.6,18.7,0.0,13.65,0.128571,15.592857
4,130220,Cassis,43.214036,5.539632,2025-10-06,12.1,20.6,0.0,16.35,0.100000,17.792857


In [31]:
df_w2 = df_w1.drop(columns=['tmin', 'tmax', 'pluie', 'td_mean'])

In [47]:
len(df_w2['ville'].value_counts())

35

In [ ]:
# save te csv file to scrapp booking 
# pd.DataFrame.to_csv(df_w2, "data/cities_weather.csv", index=False)

In [32]:
df_w3 = df_w2[df_w2['pluiew_mean'] <= 2 ]

In [33]:
df_w3.head(20)

,insee,ville,lat,lon,date,pluiew_mean,tw_mean
0,41440,Gorges du Verdon,43.749656,6.328562,2025-10-06,0.400000,10.871429
1,91960,Ariege,42.945537,1.406554,2025-10-06,1.485714,13.085714
2,110690,Carcassonne,43.213036,2.349107,2025-10-06,0.385714,17.085714
3,130010,Aix en Provence,43.529842,5.447474,2025-10-06,0.128571,15.592857
4,130220,Cassis,43.214036,5.539632,2025-10-06,0.100000,17.792857
5,130550,Marseille,43.296174,5.369953,2025-10-06,0.071429,18.007143
7,173000,La Rochelle,46.159732,-1.151595,2025-10-06,1.600000,16.085714
8,212310,Dijon,47.321581,5.041470,2025-10-06,1.671429,12.242857
10,300030,Aigues Mortes,43.566152,4.191540,2025-10-06,0.000000,17.635714
11,301890,Nimes,43.837425,4.360069,2025-10-06,0.000000,17.564286


In [34]:
df_w4 = df_w3.sort_values(by='tw_mean', ascending=False)

In [17]:
df_w5 = df_w4.head(5)

In [18]:
df_w5

,insee,ville,lat,lon,date,pluiew_mean,tw_mean
17,660530,Collioure,42.525050,3.083155,2025-10-06,0.014286,19.635714
5,130550,Marseille,43.296174,5.369953,2025-10-06,0.071429,18.007143
4,130220,Cassis,43.214036,5.539632,2025-10-06,0.100000,17.792857
10,300030,Aigues Mortes,43.566152,4.191540,2025-10-06,0.000000,17.635714
31,1309651,Saintes Maries de la mer,43.451592,4.427720,2025-10-06,0.000000,17.578571


## 3.1/ Maps Visualisation => weather param

In [20]:
import plotly.express as px

fig = px.scatter_geo(df_w5, lat="lat", lon="lon", color="tw_mean",
                     hover_name="ville", hover_data=["pluiew_mean", "tw_mean"],
                     size="tw_mean", projection="natural earth")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [47]:
import plotly.express as px

fig = px.scatter_map(df_w5, lat="lat", lon="lon", 
                     hover_name="ville", hover_data=["pluiew_mean", "tw_mean"], 
                     zoom=3, size='tw_mean' , height=300)
fig.update_layout(map_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

NameError: name 'df_w5' is not defined

# 04/ scrap booking

## 4.1/ recuperation des URL

Creation d'un spider allant directement sur booking.com et faisant une recheche sur les villes du top 5   
sur anaconda prompt pour s'assurer du lancement du spider sur le bon environnement   
* activation de l'environnement :   
-> ``` (base) C:\Users\alber> conda activate scrap_py311 ```    
* récupération du script sur le chemin de ce dernier :   
-> ``` (scrap_py311) C:\Users\alber> cd "C:\Users\alber\Desktop\visual_studio_code\dossier_jedha\Jedha_Full_stack\03_Data_Collection_&_Management_(DCM)\00- projet kayak\src" ```   
* lancement du script :   
-> ``` (scrap_py311) C:\Users\alber\Desktop\visual_studio_code\dossier_jedha\Jedha_Full_stack\03_Data_Collection_&_Management_(DCM)\00- projet kayak\src> python booking_url_hotel.py ```

## 4.2/ recupération des info par hotel via scrapy 

Creation d'un spider allant directement sur booking.com et faisant une recheche sur les villes du top 5   
sur anaconda prompt pour s'assurer du lancement du spider sur le bon environnement   
    
* activation de l'environnement :   
-> ``` (base) C:\Users\alber> conda activate scrap_py311 ```    
* récupération du script sur le chemin de ce dernier :   
-> ``` (scrap_py311) C:\Users\alber> cd "C:\Users\alber\Desktop\visual_studio_code\dossier_jedha\Jedha_Full_stack\03_Data_Collection_&_Management_(DCM)\00- projet kayak\src" ```   
* lancement du script :   
-> ``` (scrap_py311) C:\Users\alber\Desktop\visual_studio_code\dossier_jedha\Jedha_Full_stack\03_Data_Collection_&_Management_(DCM)\00- projet kayak\src> python booking_info_hotel.py ```

## 4.3/ just make key for analysts in JsonFile

In [48]:
import pandas as pd 
try:
    # essayer d'utiliser un df déjà existant
    df_w2
    print("✅ DataFrame déjà disponible en mémoire")

except NameError:
    # si df n'existe pas → on le lit depuis le CSV
    print("⚠️ DataFrame introuvable → import depuis CSV")
    df_w2 = pd.read_csv('data/cities_weather.csv')
    print(df_w2.info())


⚠️ DataFrame introuvable → import depuis CSV
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   insee        35 non-null     int64  
 1   ville        35 non-null     object 
 2   lat          35 non-null     float64
 3   lon          35 non-null     float64
 4   date         35 non-null     object 
 5   pluiew_mean  35 non-null     float64
 6   tw_mean      35 non-null     float64
dtypes: float64(4), int64(1), object(2)
memory usage: 2.0+ KB
None


In [49]:
df_w2

,insee,ville,lat,lon,date,pluiew_mean,tw_mean
0,41440,Gorges du Verdon,43.749656,6.328562,2025-10-06,0.400000,10.871429
1,91960,Ariege,42.945537,1.406554,2025-10-06,1.485714,13.085714
2,110690,Carcassonne,43.213036,2.349107,2025-10-06,0.385714,17.085714
3,130010,Aix en Provence,43.529842,5.447474,2025-10-06,0.128571,15.592857
4,130220,Cassis,43.214036,5.539632,2025-10-06,0.100000,17.792857
5,130550,Marseille,43.296174,5.369953,2025-10-06,0.071429,18.007143
6,140470,Bayeux,49.276462,-0.702474,2025-10-06,2.971429,14.364286
7,173000,La Rochelle,46.159732,-1.151595,2025-10-06,1.600000,16.085714
8,212310,Dijon,47.321581,5.041470,2025-10-06,1.671429,12.242857
9,250560,Besancon,47.238022,6.024362,2025-10-06,4.071429,11.921429


In [109]:
import json

# Charger le JSON
with open("data/hotels_details.json", "r", encoding="utf-8") as f:
    hotels = json.load(f)

# Convertir en DataFrame
df_hotels = pd.DataFrame(hotels)

# Merge avec df_top5_insee
df_merged = pd.merge(df_hotels, df_w2, on='ville', how="right")

# Reconvertir en liste de dicts
hotels_enriched = df_merged.to_dict(orient="records")

# Sauvegarder en JSON enrichi
with open("data/all_hotels_details_insee.json", "w", encoding="utf-8") as f:
    json.dump(hotels_enriched, f, indent=4, ensure_ascii=False)


# 05/data to csv to SQLtable (S3 -> ETL -> SQL)

In [51]:
import pandas as pd 
import numpy as np 
import json
import boto3

In [52]:
import dotenv
dotenv.load_dotenv()

True

## 5.1/ convert data to df to csv 

### 5.1.1/ create df with all info

#### 5.1.1.0/ make all DFs

In [90]:
with open("data/all_cities_urls_hotels.json", "r") as f:
    all_hotels_url = json.load(f)
df_all_hotels_url = pd.DataFrame(all_hotels_url)

In [110]:
with open("data/all_hotels_details_insee.json", "r") as f:
    all_hotels_details = json.load(f)
df_all_hotels_details = pd.DataFrame(all_hotels_details)

In [93]:
df_all_hotels_url.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   city    700 non-null    object
 1   url     700 non-null    object
dtypes: object(2)
memory usage: 11.1+ KB


In [111]:
df_all_hotels_details.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 689 entries, 0 to 688
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ville        689 non-null    object 
 1   url          689 non-null    object 
 2   nom          689 non-null    object 
 3   note         689 non-null    object 
 4   adresse      689 non-null    object 
 5   description  689 non-null    object 
 6   insee        689 non-null    int64  
 7   lat          689 non-null    float64
 8   lon          689 non-null    float64
 9   date         689 non-null    object 
 10  pluiew_mean  689 non-null    float64
 11  tw_mean      689 non-null    float64
dtypes: float64(4), int64(1), object(7)
memory usage: 64.7+ KB


#### 5.1.1.1/ fusion des df

In [133]:
df_h_url = df_all_hotels_url.copy(deep=True)

In [134]:
df_all_h_details = df_all_hotels_details.copy(deep=True)

In [135]:
df = pd.merge(df_h_url, df_all_h_details, on='url', how="right")

In [136]:
(df['ville']).isna().value_counts(ascending=False)

ville
False    700
Name: count, dtype: int64

#### 5.1.1.2/ clean data

In [137]:
df['note'].value_counts(ascending=False)

note
Avec une note de 9      67
Avec une note de 10     58
Avec une note de 8.5    44
Avec une note de 9.4    41
Avec une note de 8.6    39
Avec une note de 9.5    39
Avec une note de 9.1    39
Avec une note de 9.6    34
Avec une note de 9.2    33
Avec une note de 9.3    32
Avec une note de 8.2    28
Avec une note de 8.7    27
Avec une note de 8.8    26
Avec une note de 8      25
Avec une note de 8.9    20
Avec une note de 8.3    19
Avec une note de 9.7    17
Avec une note de 9.8    17
Avec une note de 8.1    16
Non disponible          14
Avec une note de 8.4    14
Avec une note de 7.9     6
Avec une note de 7.3     5
Avec une note de 7.8     4
Avec une note de 7.5     4
Avec une note de 9.9     3
Avec une note de 6       2
Avec une note de 7.7     2
Avec une note de 7.6     2
Avec une note de 6.6     2
Avec une note de 6.3     2
Avec une note de 5.5     1
Avec une note de 7.1     1
Avec une note de 7       1
Avec une note de 6.1     1
Avec une note de 5.3     1
Avec une note de 7.4   

In [138]:
df['note'] = df['note'].str.split('de ').str[-1]
df['note']

0       10
1       10
2      9.6
3      9.5
4      9.6
      ... 
695    8.6
696    8.2
697      8
698      8
699    8.4
Name: note, Length: 700, dtype: object

In [139]:
df['note'] = pd.to_numeric(df['note'], errors="coerce")

In [140]:
df['note'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 700 entries, 0 to 699
Series name: note
Non-Null Count  Dtype  
--------------  -----  
686 non-null    float64
dtypes: float64(1)
memory usage: 5.6 KB


In [141]:
df

,city,url,ville,nom,note,adresse,description,insee,lat,lon,date,pluiew_mean,tw_mean
0,Gorges du Verdon,https://www.booking.com/hotel/fr/charming-stud...,Gorges du Verdon,CHARMING studio proche centre,10.0,"243 Route des Gorges du Verdon, 04120 Castella...","Situé à Castellane, l’hébergement CHARMING stu...",41440,43.749656,6.328562,2025-10-06,0.4,10.871429
1,Gorges du Verdon,https://www.booking.com/hotel/fr/la-bastide-de...,Gorges du Verdon,La Bastide des Marcassins,10.0,"81 All. du Rossignol, 83630 Moissac-Bellevue, ...","Situé à Moissac-Bellevue, l’hébergement La Bas...",41440,43.749656,6.328562,2025-10-06,0.4,10.871429
2,Gorges du Verdon,https://www.booking.com/hotel/fr/chambres-d-ho...,Gorges du Verdon,Gorges du Verdon - Chambres d'Hotes des arches...,9.6,"Chambre des arches 2388 Rte de Chasteuil, 0412...","Situé à Castellane, l’hébergement Gorges du Ve...",41440,43.749656,6.328562,2025-10-06,0.4,10.871429
3,Gorges du Verdon,https://www.booking.com/hotel/fr/montagnac-vil...,Gorges du Verdon,Montagnac Village House - Verdon,9.5,"53 Route Nouvelle, 04500 Montagnac, France",L’hébergement Montagnac Village House - Verdon...,41440,43.749656,6.328562,2025-10-06,0.4,10.871429
4,Gorges du Verdon,https://www.booking.com/hotel/fr/le-petit-para...,Gorges du Verdon,Le Petit Paradis Provençal,9.6,"261C Imp. du Bosquet, 04500 Allemagne-en-Prove...",Possédant une piscine extérieure ouverte en sa...,41440,43.749656,6.328562,2025-10-06,0.4,10.871429
...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,Biarritz,https://www.booking.com/hotel/fr/radisson-blu-...,Biarritz,Le Talaia Hôtel & Spa Biarritz - MGallery Coll...,8.6,"1 Carrefour Helianthe, 64200 Biarritz, France","Face à l'océan Atlantique, le Talaia proposera...",6412251,43.483252,-1.559278,2025-10-06,2.1,17.135714
696,Biarritz,https://www.booking.com/hotel/fr/utopy-hostel-...,Biarritz,GARDEN - Utopy Hostel,8.2,"Batiment E 27 Avenue de Migron, 64200 Biarritz...","Situé à Biarritz, l’établissement GARDEN - Uto...",6412251,43.483252,-1.559278,2025-10-06,2.1,17.135714
697,Biarritz,https://www.booking.com/hotel/fr/amaryshotel.f...,Biarritz,Sure Hotel by Best Western Biarritz Aeroport,8.0,"24 boulevard marcel dassault, 64200 Biarritz, ...",Le Sure Hotel by Best Western Biarritz Aeropor...,6412251,43.483252,-1.559278,2025-10-06,2.1,17.135714
698,Biarritz,https://www.booking.com/hotel/fr/baron-de-biar...,Biarritz,Hôtel Parc Mazon-Biarritz,8.0,"13 avenue Maréchal Joffre , 64200 Biarritz, Fr...","Situé dans le centre de Biarritz, à 450 mètres...",6412251,43.483252,-1.559278,2025-10-06,2.1,17.135714


In [142]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   city         700 non-null    object 
 1   url          700 non-null    object 
 2   ville        700 non-null    object 
 3   nom          700 non-null    object 
 4   note         686 non-null    float64
 5   adresse      700 non-null    object 
 6   description  700 non-null    object 
 7   insee        700 non-null    int64  
 8   lat          700 non-null    float64
 9   lon          700 non-null    float64
 10  date         700 non-null    object 
 11  pluiew_mean  700 non-null    float64
 12  tw_mean      700 non-null    float64
dtypes: float64(5), int64(1), object(7)
memory usage: 71.2+ KB


In [143]:
df = df.drop(columns='city')

In [144]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   url          700 non-null    object 
 1   ville        700 non-null    object 
 2   nom          700 non-null    object 
 3   note         686 non-null    float64
 4   adresse      700 non-null    object 
 5   description  700 non-null    object 
 6   insee        700 non-null    int64  
 7   lat          700 non-null    float64
 8   lon          700 non-null    float64
 9   date         700 non-null    object 
 10  pluiew_mean  700 non-null    float64
 11  tw_mean      700 non-null    float64
dtypes: float64(5), int64(1), object(6)
memory usage: 65.8+ KB


### 5.1.2/ get lat & lon by address

In [145]:
import requests
import os
import dotenv

In [146]:
# Charger le fichier .env
dotenv.load_dotenv()

# IMPORTANT : Récupérer la variable depuis les variables d'environnement
HERE_api_key = os.getenv("HERE_api_key")
HERE_api_URL = os.getenv("HERE_api_URL")  # Si vous l'avez aussi dans le .env

# Vérification (optionnel)
print(f"API Key chargée : {HERE_api_key[:10]}..." if HERE_api_key else "❌ Erreur : clé non trouvée")

API Key chargée : E7niaCI6QR...


In [147]:
# Fonction qui renvoie (lat, lon)
def geocode_here(address):
    params = {"q": address, "apiKey": HERE_api_key}
    r = requests.get(HERE_api_URL, params=params).json()
    if "items" in r and len(r["items"]) > 0:
        lat_h = r["items"][0]["position"]["lat"]
        lon_h = r["items"][0]["position"]["lng"]
        return lat_h, lon_h
    else:
        return None, None

In [ ]:
# Utilisation avec apply
# df[["lat_h", "lon_h"]] = df["adresse"].apply(lambda x: pd.Series(geocode_here(x)))

In [149]:
df

,url,ville,nom,note,adresse,description,insee,lat,lon,date,pluiew_mean,tw_mean,lat_h,lon_h
0,https://www.booking.com/hotel/fr/charming-stud...,Gorges du Verdon,CHARMING studio proche centre,10.0,"243 Route des Gorges du Verdon, 04120 Castella...","Situé à Castellane, l’hébergement CHARMING stu...",41440,43.749656,6.328562,2025-10-06,0.4,10.871429,43.84507,6.50694
1,https://www.booking.com/hotel/fr/la-bastide-de...,Gorges du Verdon,La Bastide des Marcassins,10.0,"81 All. du Rossignol, 83630 Moissac-Bellevue, ...","Situé à Moissac-Bellevue, l’hébergement La Bas...",41440,43.749656,6.328562,2025-10-06,0.4,10.871429,43.65037,6.18103
2,https://www.booking.com/hotel/fr/chambres-d-ho...,Gorges du Verdon,Gorges du Verdon - Chambres d'Hotes des arches...,9.6,"Chambre des arches 2388 Rte de Chasteuil, 0412...","Situé à Castellane, l’hébergement Gorges du Ve...",41440,43.749656,6.328562,2025-10-06,0.4,10.871429,43.83677,6.42174
3,https://www.booking.com/hotel/fr/montagnac-vil...,Gorges du Verdon,Montagnac Village House - Verdon,9.5,"53 Route Nouvelle, 04500 Montagnac, France",L’hébergement Montagnac Village House - Verdon...,41440,43.749656,6.328562,2025-10-06,0.4,10.871429,43.77882,6.09671
4,https://www.booking.com/hotel/fr/le-petit-para...,Gorges du Verdon,Le Petit Paradis Provençal,9.6,"261C Imp. du Bosquet, 04500 Allemagne-en-Prove...",Possédant une piscine extérieure ouverte en sa...,41440,43.749656,6.328562,2025-10-06,0.4,10.871429,43.78818,6.01919
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,https://www.booking.com/hotel/fr/radisson-blu-...,Biarritz,Le Talaia Hôtel & Spa Biarritz - MGallery Coll...,8.6,"1 Carrefour Helianthe, 64200 Biarritz, France","Face à l'océan Atlantique, le Talaia proposera...",6412251,43.483252,-1.559278,2025-10-06,2.1,17.135714,43.47829,-1.56457
696,https://www.booking.com/hotel/fr/utopy-hostel-...,Biarritz,GARDEN - Utopy Hostel,8.2,"Batiment E 27 Avenue de Migron, 64200 Biarritz...","Situé à Biarritz, l’établissement GARDEN - Uto...",6412251,43.483252,-1.559278,2025-10-06,2.1,17.135714,43.47374,-1.55014
697,https://www.booking.com/hotel/fr/amaryshotel.f...,Biarritz,Sure Hotel by Best Western Biarritz Aeroport,8.0,"24 boulevard marcel dassault, 64200 Biarritz, ...",Le Sure Hotel by Best Western Biarritz Aeropor...,6412251,43.483252,-1.559278,2025-10-06,2.1,17.135714,43.47177,-1.53536
698,https://www.booking.com/hotel/fr/baron-de-biar...,Biarritz,Hôtel Parc Mazon-Biarritz,8.0,"13 avenue Maréchal Joffre , 64200 Biarritz, Fr...","Situé dans le centre de Biarritz, à 450 mètres...",6412251,43.483252,-1.559278,2025-10-06,2.1,17.135714,43.47776,-1.56218


In [150]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   url          700 non-null    object 
 1   ville        700 non-null    object 
 2   nom          700 non-null    object 
 3   note         686 non-null    float64
 4   adresse      700 non-null    object 
 5   description  700 non-null    object 
 6   insee        700 non-null    int64  
 7   lat          700 non-null    float64
 8   lon          700 non-null    float64
 9   date         700 non-null    object 
 10  pluiew_mean  700 non-null    float64
 11  tw_mean      700 non-null    float64
 12  lat_h        698 non-null    float64
 13  lon_h        698 non-null    float64
dtypes: float64(7), int64(1), object(6)
memory usage: 76.7+ KB


### 5.1.3/ make visualisation & csv

In [151]:
import plotly.express as px

fig = px.scatter_map(df, lat="lat_h", lon="lon_h", color="note",
                     color_continuous_scale="Turbo",
                     hover_name="nom", hover_data=["url", "adresse"],
                     zoom=8, height=300)
fig.update_traces(marker_size=10)
fig.update_layout(map_style="basic")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [168]:
import plotly.express as px

fig = px.scatter_map(df, lat="lat", lon="lon", color="tw_mean",
                     color_continuous_scale="Turbo",
                     hover_name="nom", hover_data=["url", "adresse"],
                     zoom=8, height=300)
fig.update_traces(marker_size=10)
fig.update_layout(map_style="basic")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
# pd.DataFrame.to_csv(df, "data/hotels_info.csv", index=False)

## 5.2/ AWS S3 bucket => ETL => NeonDB 

### 5.2.1/ AWS S3 Bucket

In [153]:
import boto3
import os 
import dotenv

dotenv.load_dotenv()
AWS_acces_key = os.getenv('AWS_acces_key')
AWS_secret_acces_key = os.getenv('AWS_secret_acces_key')

In [154]:
s3_client = boto3.client(
    's3',
    aws_access_key_id=os.getenv('AWS_acces_key'),
    aws_secret_access_key=os.getenv('AWS_secret_acces_key'),
    region_name=os.getenv('AWS_REGION', 'eu-west-3')
)
bucket_name = os.getenv('S3_BUCKET_NAME')

In [155]:
s3_client.upload_file('data/hotels_info.csv', bucket_name, 'hotels_info.csv')

### 5.2.2/ ETL 

In [156]:
import pandas as pd
import boto3
import os 
from sqlalchemy import create_engine, text
from dotenv import load_dotenv


In [157]:
s3_client = boto3.client(
    's3',
    aws_access_key_id=os.getenv('AWS_acces_key'),
    aws_secret_access_key=os.getenv('AWS_secret_acces_key'),
    region_name=os.getenv('AWS_REGION', 'eu-west-3')
)
bucket_name = os.getenv('S3_BUCKET_NAME')
NEON_db = os.getenv('NEON_URI')

In [161]:
Object_s3 = s3_client.get_object(Bucket=bucket_name, Key='hotels_info.csv')

In [162]:
engine = create_engine(NEON_db, echo=True)

In [163]:
df_test = pd.read_csv(Object_s3['Body'])
df_test.to_sql("hotels_info", engine, if_exists="replace", index=True)

2025-10-04 01:59:49,692 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-10-04 01:59:49,693 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-10-04 01:59:49,747 INFO sqlalchemy.engine.Engine select current_schema()
2025-10-04 01:59:49,748 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-10-04 01:59:49,800 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-10-04 01:59:49,800 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-10-04 01:59:49,850 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-10-04 01:59:49,854 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname

700